# Coursework 3: Bilinear Inverse Problems and Low-Rank Matrix Recovery

[] By tick the checkbox, we hereby declare that this coursework report is our own and autonomous work. We have acknowledged all material and sources used in its preparation, including books, articles, reports, lecture notes, internet software packages, and any other kind of document, electronic or personal communication. This work has not been submitted for any other assessment.

## 3.1 Test Data Generation (10%)

We consider the low-rank matrix completion problem given by 
$$
    \bm{y} = \mathcal{P}_{\Omega}(\bm{X}) 
$$
where $\bm{X} \in \mathbb{R}^{m \times n}$ is a low rank matrix of rank $r$. 

Data generation: Write $\bm{X} = \bm{U} \bm{G} \bm{V}^{\mathsf{T}}$, where $\bm{U} \in \mathbb{R}^{m \times r}$, $\bm{G} \in \mathbb{R}^{r \times r}$, and $\bm{V} \in \mathbb{R}^{n \times r}$ are matrices with i.i.d. $\mathcal{N}(0,1)$ Gaussian entries. (Note that by $\bm{X} = \bm{U} \bm{G} \bm{V}^{\mathsf{T}}$ we are not talking about SVD.)

Design and implement a function `LRMC_data_gen` to generate test data. Provide necessary documentation.

In [2]:
import Pkg
Pkg.add("Distributions")

    Updating registry at `C:\Users\andre\.julia\registries\General`
    Updating git-repo `https://github.com/JuliaRegistries/General.git`
   Resolving package versions...
   Installed OpenSpecFun_jll ── v0.5.5+0
   Installed Libffi_jll ─────── v3.2.2+1
   Installed Rmath_jll ──────── v0.3.0+0
   Installed FillArrays ─────── v0.12.7
   Installed DensityInterface ─ v0.4.0
   Installed StatsFuns ──────── v0.9.14
   Installed QuadGK ─────────── v2.4.2
   Installed PDMats ─────────── v0.11.5
   Installed Rmath ──────────── v0.7.0
   Installed SpecialFunctions ─ v1.8.1
   Installed Distributions ──── v0.25.32
    Updating `C:\Users\andre\.julia\environments\v1.6\Project.toml`
  [31c24e10] + Distributions v0.25.32
    Updating `C:\Users\andre\.julia\environments\v1.6\Manifest.toml`
  [b429d917] + DensityInterface v0.4.0
  [31c24e10] + Distributions v0.25.32
  [1a297f60] + FillArrays v0.12.7
  [90014a1f] + PDMats v0.11.5
  [1fd47b50] + QuadGK v2.4.2
  [79098fc4] + Rmath v0.7.0
  [276daf66] + 

In [2]:
using Random
using LinearAlgebra
using StatsBase
using Distributions
using Plots

function GaussianGen(m::Int64, n::Int64)
    A = randn(Float64, m, n)
    Norm = zeros(m,n)
    for i = 1:n
        Norm[:,i] = normalize(A[:,i], 2);
    end
    return Norm
end

function Observation_samples(X::AbstractArray, m::Int64, n::Int64, samples::Int64)

    random_indices = sample(randperm(m*n), samples, replace=false)

    Ω = zeros(Bool, m, n)
    Ω[random_indices] .= true

    Y = zeros(m, n)

    Y[Ω] = X[Ω]

    return Y, Ω
end

function LRMC_data_gen(m::Int64, n::Int64, r::Int64)
  U = GaussianGen(m, r)
  G = GaussianGen(r, r)
  V = GaussianGen(n, r)

  X = U * G * V'

  return X
end

LRMC_data_gen (generic function with 1 method)

## 3.2 Matrix Completion Techniques

In the following, the suggested simulation setup is that $m = 32$, $n=48$, $r$ varies in $2:2:8$, and $|\Omega|/mn$ varies in $\{1/8,~ 1/6,~ 1/4,~ 1/2\}$. 

### 3.2.1 Alternating Minimization (20%)

Design, implement, and run tests for the alternating minimization method for low-rank matrix completion. Use the function name `LRMCRec_AM`. Provide necessary documentation.

### 3.2.2 Iterative Hard Thresholding (IHT) (20%)

Design, implement, and run simple tests for the IHT algorithm for low-rank matrix completion. Use the function name `LRMCRec_IHT`. Provide necessary documentation. 

### 3.2.3 Iterative Shrinkage-Thresholding Algorithm (ISTA) (25%)

Design, implement, and run simple tests for ISTA (to solve the Lasso formulation) for low-rank matrix completion. Use the function name `LRMCRec_ISTA`. Provide necessary documentation. Use simulations to discuss the choice of parameters.

### 3.2.4 Lasso-ADMM (25%)

Design, implement, and run simple tests for an ADMM algorithm (to solve the Lasso formulation) for low-rank matrix completion. Use the function name `LRMCRec_ADMM`. Provide necessary documentation. Compare ADMM and ISTA in terms of convergence.

Lasso formulation is written as:
$$
argmin_{x} \frac{1}{2}||y - Ax||_{2}^{2} + \lambda||x||_{1}
$$
Since ADMM algorithm will be used for low rank matrix completion, we can interpret the $Ax$ of the lasso problem as the linear mapping of the revealed entries of x (our estimation matrix) indexed by $\Omega$.


Given that the corresponding ADMM iterations are:
$$
x^{k+1} = arg min_x \frac{1}{2}||y - Ax||_{2}^{2} + \frac{\rho}{2}||x - z_{k} + \frac{v_{k}}{\rho}||_{2}^{2}
$$
$$
z^{k+1} = arg min_z \lambda ||z||_{1} + \frac{\rho}{2}||x_{k+1} - z + \frac{v_{k}}{\rho}||_{2}^{2}
$$
$$
v^{k+1} = v_{k} + \rho (x^{k+1} - z^{k+1})
$$

For $x^{k+1}$, we can calculate the closed form solutions by setting $\nabla f(x)=0$
$$
x = (A^{T}A + \rho)^{-1} (A^{T}y + \rho z^{k} - v^{k})
$$
We can do the same for $z^{k+1}$. We just need to be wary of the sign(z) that appears when differentiating the l1 norm.
$$
\lambda sign(z) + \rho z = \rho x^{k+1} + v^{k}
$$

When $z>0$ :
$$
\rho z = \rho x^{k+1} + v^{k} - \lambda
$$
$$
z = x^{k+1} + \frac{v^{k}}{\rho} - \frac{\lambda}{\rho}
$$

When $z<0$:
$$
z = x^{k+1} + \frac{v^{k}}{\rho} + \frac{\lambda}{\rho}
$$

In [94]:
using LinearAlgebra
using SparseArrays

#=
For solving lasso problem with admm:

    0.5(y-Ax)^2 + lambda*norm(x,1)
    
=#

function nz_idx(Y)
    idxs = []
    for (i,v) in enumerate(Y)
        if Y[i] != 0
            push!(idxs, i)
        end
    end
    idxs = Array{Int64}(idxs)
    return idxs
end

function get_mapping_matrix(Y)
    m = size(Y, 1)
    n = size(Y, 2)

    c = m*n
    r = length(nz_idx(Y))
    A = zeros(r,c)

    index_set = nz_idx(Y)

    for (i,v) in enumerate(index_set)
       A[i,v] = 1  #create identity matrix that acts as linear operator
    end

    return A
end

function LRMCRec_ADMM(Y, lambda, rho)
    maxiter = 500
    loss = []

    A = get_mapping_matrix(Y)

    c = size(Y,2)
    r = size(A,2)

    identity = sparse(I,size(A,2),size(A,2))
    # #Initialise
    
    # z = randn(r,c)
    # v = zeros(size(Y,1), size(Y,2))
    # x = 0

    # for i in 1:maxiter
    #     x = inv(A'*A + rho * identity) * (A'*Y + rho * z - v)

    #     if z >= 0
    #         z = x + v/rho - lambda/rho
    #     end
    #     if z < 0
    #         z = x + v/rho + lambda/rho
    #     end
    #     v = v + rho * (x - z)
        
    #     cost = 0.5 * norm(Y - A*x, 2) + lambda * norm(x, 1)
    #     append!(loss, cost)
    # end
    return identity
end
    

LRMCRec_ADMM (generic function with 2 methods)

In [95]:
m = 48; n = 32
#Ω_sizes = [192,256,384,768]
Ω_size = 192
plot_arr = Plots.Plot{Plots.GRBackend}[]
lambda = 0.001
rho = 1e-4
r = 2
Y, omega = Observation_samples(X, m, n, Ω_size)
get_mapping_matrix(Y)
LRMCRec_ADMM(Y,lambda,rho)

# for r= 2:2:8 
#     p_r = plot()
#     X  = LRMC_data_gen(m, n, r)
#     for Ω_size in Ω_sizes 
#         Y, omega = Observation_samples(X, m, n, Ω_size)
#         x, loss = LRMCRec_ADMM(Y,lambda,rho)
#         plot!(p_r, num_iters, loss, label="|Ω|=$(Ω_size)", title="IHT for r=$(r)",fmt = :png,dpi=2080)
#         @show rank(x)
#     end
#     push!(plot_arr,p_r)
# end

#plot(plot_arr[1],plot_arr[2],plot_arr[3],plot_arr[4])

1536×1536 SparseMatrixCSC{Bool, Int64} with 1536 stored entries:
⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄

## Highlight

Please list a couple of highlights of your coursework that may impress your markers.